# TITLE

In [1]:
from scipy.spatial import cKDTree
import numpy as np
import pandas as pd
import random

# PARAMETERS

In [2]:
#Parameters
N=10; #songs in the playlist
method = 1 # 0 -> KD-Tree | 1 -> 2D graphs coords | 2 -> Heat diffusion

# Paylists
meta_playlist = list()

# Data Loading

In [3]:
#Load data
dataSet = pd.read_pickle('data/dataSet.csv')
coords2D = np.load("data/coords2D.npy").item()
dfmax=np.load('data/dfmax.npy')
tracks = pd.read_csv('data/tracks.csv', index_col=0, header=[0, 1]);
tracks_raw = pd.read_csv('../data/fma_metadata/raw_tracks.csv', index_col=0);

In [4]:
# Loading custom songs features
features_c = pd.read_csv('data/custom_features.csv', index_col=0, header=[0, 1, 2], sep=',', encoding='utf-8')
tid_c = pd.read_csv('data/custom_tid.csv', index_col=0, header=[0], sep=',', encoding='utf-8')

features_c=features_c.divide(dfmax)
numb_cust=len(features_c)
features_c.head()

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
-1            0.000040 -0.000210 -0.000410 -0.000130  0.000480  0.000845   
-2            0.000801  0.001783  0.000376  0.000315  0.000443  0.000175   
-3           -0.000090 -0.000168 -0.000703 -0.000249 -0.000190 -0.000294   
-4            0.000045 -0.000121  0.000470 -0.000084 -0.000264 -0.000272   
-5            0.000034 -0.000189  0.000169  0.000012 -0.000119 -0.000222   

feature                                               ...      tonnetz  \
statistics                                            ...          std   
number            07        08        09        10    ...           04   
track_id                                              ...                
-1          0.000117 -0.000020  0.000193 -0.000021    ...     0.283257   
-2          0.000327  0.000692  0.000757  0.000758    ...     0.168253   
-3         -0.000254  0.000211  0.000168 -0.000311    ...     0.391784   
-4          0.000211  0.000181  0.000106 -0.000429    ...     0.350125   
-5          0.000022  0.000011  0.000394 -0.000348    ...     0.279832   

feature                              zcr                                \
statistics                      kurtosis       max      mean    median   
number            05        06        01        01        01        01   
track_id                                                                 
-1          0.201160  0.127691  0.000817  0.393258  0.083793  0.074114   
-2          0.135252  0.101407  0.000683  0.459697  0.097202  0.079564   
-3          0.238435  0.219134  0.006972  0.660479  0.043547  0.028883   
-4          0.251780  0.175684  0.008211  0.616512  0.049449  0.040327   
-5          0.172038  0.176377  0.008916  0.340010  0.028089  0.021798   

feature                                   
statistics       min      skew       std  
number            01        01        01  
track_id                                  
-1          0.000000  0.025611  0.110382  
-2          0.000000  0.026778  0.144958  
-3          0.018622  0.091880  0.143848  
-4          0.013035  0.075646  0.083336  
-5          0.000000  0.091840  0.057087  

[5 rows x 518 columns]

# Functions

In [5]:
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, 'Open')+'&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp'+'<a target="_blank" href="{}">{}</a>'.format(val+'/download', 'Download')

In [6]:
def display_result(playlist):
    result=pd.DataFrame({'Artist': tracks.artist.name[playlist], \
                         'Title':tracks.track.title[playlist], \
                         'Links':tracks_raw['track_url'][playlist]}) 

    # Output 
    result.reset_index(inplace=True)
    result.drop(columns='track_id',inplace=True)
    result = result.style.format({'Links':make_clickable})
    
    # Output
    display(result)

# KD-Tree

In [7]:
# Convert to np.array
points_ref = np.array([tuple(x) for x in dataSet.drop(columns='label',level=0).values])

# Construct the KD Tree
tree = cKDTree(points_ref)

## KD-Tree based

In [8]:
# Convert to np.array
features_c_t = np.array([tuple(x) for x in features_c.values])

# Find the 10th nearest points with Euclidean norm in the tree
dist, idx = tree.query(features_c_t, k=N, p=2)

In [9]:
#list of the tracks nearest to each custom song
nearest_tracks=list()

over_playlist = list()
# For every added song we fetch from the query
for tid, nodes in enumerate(idx):

    # From the neighbours nodes list, retrieve their track id, artist and title
    track_list = list(dataSet.iloc[list(nodes)].index.values)
    
    # Returns
    nearest_tracks.append(track_list[0]) # needed
    playlist = random.sample(track_list,N)
    over_playlist.append(playlist)
    
# 
meta_playlist.append(over_playlist)

## 2-D coords based

In [10]:
# Create the KD-Tree from the graphs coordinates
graphCoords=list()
for i in coords2D:
    graphCoords.append(tuple(coords2D[i]))
graphCoords=np.array(graphCoords)
graphTree = cKDTree(graphCoords)

In [11]:
over_playlist = list()

# For each playlist, access nearest tracks
for i in tid_c['Playlist name'].unique():
    
    playlist=list()
    # Find closest tracks in the graph for a given custom song
    for i in nearest_tracks:
        dist, idx = graphTree.query(tuple(coords2D[i]), k=np.ceil(N/numb_cust), p=2)
        playlist.append([list(coords2D.keys())[i] for i in idx])

    # Select in the closest neighbours N random tracks
    playlist=[y for x in playlist for y in x]
    playlist=random.sample(playlist,N) # Returns

    # Add to meta playlist
    over_playlist.append(playlist)
    
#
meta_playlist.append(over_playlist)

# Output

In [12]:
for i,playlist in enumerate(tid_c['Playlist name'].unique()):
    print('Playlist:', playlist)
    display_result(meta_playlist[method][i])

Playlist: Tim


,Artist,Title,Links
0,Krestovsky,"A Sure Thing (""wedding song"")",Open Download
1,manuel gordiani,imensa prece,Open Download
2,Heavy Links,Reactivated,Open Download
3,Ulises Lima,Color Blindness,Open Download
4,Son Altesse Furieuse,Relax,Open Download
5,Half Japanese,Too Much Fun,Open Download
6,Jonah Rapino,1. Solo Project,Open Download
7,Derek Clegg,Easy,Open Download
8,Fallen to Flux,Relapse,Open Download
9,AWOL,Food,Open Download


Playlist: Anael


,Artist,Title,Links
0,manuel gordiani,imensa prece,Open Download
1,Jonah Rapino,1. Solo Project,Open Download
2,Fallen to Flux,Relapse,Open Download
3,AWOL,Food,Open Download
4,Krestovsky,"A Sure Thing (""wedding song"")",Open Download
5,Half Japanese,Too Much Fun,Open Download
6,Upsilon Acrux,So Thereby...and furthermore...thus henceforth...Mono,Open Download
7,Derek Clegg,Easy,Open Download
8,Heavy Links,Reactivated,Open Download
9,Jonathan Coulton,I Feel Fantastic,Open Download


Playlist: Lucas


,Artist,Title,Links
0,Half Japanese,Too Much Fun,Open Download
1,Son Altesse Furieuse,Relax,Open Download
2,manuel gordiani,imensa prece,Open Download
3,Jonah Rapino,1. Solo Project,Open Download
4,Derek Clegg,All of My Love Resides in Paris,Open Download
5,Heavy Links,Reactivated,Open Download
6,Krestovsky,"A Sure Thing (""wedding song"")",Open Download
7,Fallen to Flux,Relapse,Open Download
8,Upsilon Acrux,So Thereby...and furthermore...thus henceforth...Mono,Open Download
9,Jonathan Coulton,I Feel Fantastic,Open Download


Playlist: Joacqim


,Artist,Title,Links
0,Ulises Lima,Color Blindness,Open Download
1,Fallen to Flux,Relapse,Open Download
2,Jonah Rapino,1. Solo Project,Open Download
3,Heavy Links,Reactivated,Open Download
4,Derek Clegg,All of My Love Resides in Paris,Open Download
5,Upsilon Acrux,So Thereby...and furthermore...thus henceforth...Mono,Open Download
6,Derek Clegg,Easy,Open Download
7,Krestovsky,"A Sure Thing (""wedding song"")",Open Download
8,AWOL,Food,Open Download
9,manuel gordiani,imensa prece,Open Download
